# Assessing histones as a model system

## Setup

In [76]:
import os
import random
import matplotlib.pyplot as plt
import pandas as pd
from localcider.sequenceParameters import SequenceParameters
from Bio import SeqIO
from collections import defaultdict

## Sequence analysis using LocalCider

Documentation on parameters from the Pappu lab can be found here:
- [CIDER](http://pappulab.wustl.edu/CIDER/about/)
- [localCIDER](http://pappulab.github.io/localCIDER/)

In [77]:
# Loading data
data_dir = '../../data/histones'
histones = {}
for filename in os.listdir(data_dir):
    filepath = '/'.join([data_dir, filename])
    for record in SeqIO.parse(open(filepath, 'r'), 'fasta'):
        name = record.description[19:].split()[0] + ' ' + record.description.split()[-1]
        histones[name] = str(record.seq)

print("The following proteins are investigated:")
for id, seq in histones.items():
    print(f" - {id}")

The following proteins are investigated:
 - H1.3 CTD
 - H1.2 CTD
 - H3.1t NTD
 - H2B NTD
 - H1.4 CTD
 - H1.1 CTD
 - H1.0 CTD


In [78]:
# Defining analysis
def cider_parameters(seq):
    """
    Takes a sequence, returns a dictionary of its CIDER parameters. 
    """
    # Creating a SequenceParameters object from sequence
    SeqOb = SequenceParameters(seq)

    # Calculating parameters
    params = {}
    params['kappa'] = SeqOb.get_kappa()
    params['FCR'] = SeqOb.get_FCR()
    params['NCPR'] = SeqOb.get_NCPR()
    params['Hydrophobicity'] = SeqOb.get_mean_hydropathy()
    params['Frac. dis. prom.'] = SeqOb.get_fraction_disorder_promoting()

    return params

def cider_parameters_group(proteins):
    """ 
    Takes a dict of proteins sequences, returns a dataframe with CIDER parameters
    """
    # Collecting parameters
    params_list = []
    for tag, seq in proteins.items():
        params_list.append(cider_parameters(seq))

    # Combining into datamframe
    params = defaultdict(list)
    for d in params_list:
        for key, value in d.items():
            params[key].append(value)
    params = pd.DataFrame(params, index=proteins.keys())

    return params

def cider_positionwise_plot(seq, windowsize=5):
    """
    Takes a sequence, returns fig, ax objects of a plot of its positionwise window-averaged CIDER parameters.
    """
    # Creating a SequenceParameters object from sequence
    SeqOb = SequenceParameters(seq)

    # Plotting positionwise CIDER parameters
    fig, ax = plt.subplots()
    ax.plot(*SeqOb.get_linear_NCPR(windowsize), label="NCPR", color='orange')

    # Decorating
    ax.hlines(0,0,SeqOb.get_length(), ls='dashed', color='k', alpha=0.2)
    ax.set_ylim(-1,1)
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.spines['left'].set_linewidth(2)
    ax.spines['bottom'].set_linewidth(2)
    ax.xaxis.set_tick_params(width=2)
    ax.yaxis.set_tick_params(width=2)
    ax.set_xlabel("Position")
    ax.legend()

    return fig, ax



In [79]:
# Performing sequence analysis using localCIDER
params = cider_parameters_group(histones)
print(params)

              kappa       FCR      NCPR  Hydrophobicity  Frac. dis. prom.
H1.3 CTD   0.147564  0.363636  0.348485        3.375000          0.916667
H1.2 CTD   0.155614  0.368852  0.352459        3.515574          0.893443
H3.1t NTD  0.111152  0.325581  0.325581        3.232558          0.883721
H2B NTD    0.301676  0.485714  0.371429        2.614286          0.942857
H1.4 CTD   0.148240  0.375000  0.359375        3.391406          0.953125
H1.1 CTD   0.130636  0.360656  0.344262        3.388525          0.885246
H1.0 CTD   0.154432  0.432432  0.378378        3.249550          0.891892


## Generating sequences for CALVADOS simulation

To assess whether histones could serve as a model system, we want to see how their radius of gyration $R_g$ changes dependening on the assembly of their sequences. To do so, three sequences are generated from the original one:
- A wildtype sequence (*wt*)
- A sequence with positive and negative charges clustered in the C- and N-terminal respectively (*clustered*)
- A randomly shuffled sequence (*shuffled*)

In [48]:
# Defining algorithms

def cluster_charges(seq, ngroup, cgroup, seed, mc_threshold=1):
    """
    Takes a sequence, clusters two groups of distinct amino acids in oppposite ends by switiching residue positions in the sequence
    Optional parameter for whether or not to do Monte Carlo criteria for switching (1 = 100% chance of switching to cluster)
    """
    # Prepping
    random.seed(seed)
    seq = list(seq)

    # Checking groups
    assert set(ngroup) != set(cgroup), "The groups must have any amino acids in common!"

    # Finding positions of N-terminal group residues
    i_ngroup = [i for i, aa in enumerate(seq) if aa in ngroup]
    
    # Loop over sequence for N-terminal group
    for i, aa in enumerate(seq):

        # If group residue, pass and remove group residue from indexed list (downsteam residues)
        if aa in ngroup:
            i_ngroup.remove(i)

        # If not, switch with downstream group residue
        else:
            
            # Monte Carlo criteria
            if random.random() < mc_threshold:

                # Switch residue with random downstream group residue
                j = random.choice(i_ngroup)
                seq[i], seq[j] = seq[j], seq[i]
                i_ngroup.remove(j)
        
        # If all downstream group residues are sorted, stop
        if len(i_ngroup) == 0:
            break

    # Reversing sequence and finding positions of C-terminal group residues
    seq.reverse()
    i_cgroup = [i for i, aa in enumerate(seq) if aa in cgroup]

    # Reverse sequence and loop over it for C-terminal group
    for i, aa in enumerate(seq):

        # If group residue, pass and remove group residue from indexed list (downsteam residues)
        if aa in cgroup:
            i_cgroup.remove(i)

        # If not, switch with downstream group residue
        else:
            
            # Monte Carlo criteria
            if random.random() < mc_threshold:

                # Switch residue with random downstream group residue
                j = random.choice(i_cgroup)
                seq[i], seq[j] = seq[j], seq[i]
                i_cgroup.remove(j)
        
        # If all downstream group residues are sorted, stop
        if len(i_cgroup) == 0:
            break

    # Reverse and join sequence
    seq.reverse()
    seq = ''.join(seq)

    return seq

def shuffle(seq, seed):
    """
    Takes a sequence and shuffles it randomly.
    """
    # Prepping
    seq = list(seq)

    # Shuffling
    random.seed(seed)
    random.shuffle(seq)

    # Joining
    seq = ''.join(seq)

    return seq

In [73]:
# Generating sequences
name = "H2B NTD"
print("Choosen histone:", name)
seq = histones[name]
seed = random.randint(0,100) #300700

variants = {"Wild-type":            seq,
            "Clustered charges":    cluster_charges(seq, ['K', 'R'], ['D', 'E'], seed, mc_threshold=1),
            "Random shuffle":       shuffle(seq, seed)}

# Analysis
print(cider_parameters_group(variants))


Choosen histone: H2B NTD
                      kappa       FCR      NCPR  Hydrophobicity  \
Wild-type          0.301676  0.485714  0.371429        2.614286   
Clustered charges  0.965685  0.485714  0.371429        2.614286   
Random shuffle     0.247571  0.485714  0.371429        2.614286   

                   Frac. dis. prom.  
Wild-type                  0.942857  
Clustered charges          0.942857  
Random shuffle             0.942857  
